In [1]:
import numpy as np
import csv
import random
import pandas as pd
import copy

配列
```
[2, 3, 4, 1, 5]
```
は以下のようなQueenの配置を示す.

|   | 1 | 2 | 3 | 4 | 5 | 
|---|---|---|---|---|---|
| 1 |   | Q |   |   |   |
| 2 |   |   | Q |   |   |
| 3 |   |   |   | Q |   |
| 4 | Q |   |   |   |   |
| 5 |   |   |   |   | Q |

つまり、i番目の配列の中身がjのときQueenの場所は(i, j)(i行j列目)に存在する

# Generate Queen-size Array

In [2]:
queen_size = 5
population = 6
arr = np.array([
    np.array([i for i in range(1,queen_size+1)])
    for j in range(population)
])
print(arr)

for i, _arr in enumerate(arr):
    np.random.shuffle(_arr)
    arr[i] = _arr
print(arr)

[[1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]]
[[3 4 5 2 1]
 [2 5 4 3 1]
 [3 5 1 4 2]
 [3 5 2 4 1]
 [2 4 1 5 3]
 [3 5 4 1 2]]


## evaluate
|   | 1 | 2 | 3 | 4 | 5 | 
|---|---|---|---|---|---|
| 1 |   | Q |   |   |   |
| 2 |   |   | Q |   |   |
| 3 |   |   |   | Q |   |
| 4 | Q |   |   |   |   |
| 5 |   |   |   |   | Q |

In [3]:
tmp_arr = np.array([2, 3, 4, 1, 5])

In [4]:
def get_penalty(arr):
    penalty = 0
    for i in range(arr.shape[0]):
        for j in range(1, arr.shape[0]):
            if i+j < 5:  # 一番下を制限
                if arr[i] - 1 + j < 5 and arr[i+j] == arr[i]+j:  # 右下
                    penalty += 1
                    #print("i+j:",i,j)
                if arr[i] - 1 - j >= 0 and arr[i+j] == arr[i]-j:  # 左下
                    penalty += 1
                    #print("i+j:",i,j)
            if i-j >= 0:
                if arr[i] - 1 + j < 5 and arr[i-j] == arr[i]+j:  # 右上
                    penalty += 1
                    #print("i-j:",i,j)
                if arr[i] - 1 - j >= 0 and arr[i-j] == arr[i]-j:  # 左上
                    penalty += 1
                    #print("i-j:",i,j)
    penalty = int(penalty / 2)
    return penalty

In [5]:
get_penalty(tmp_arr)

4

In [6]:
def evaluate_queen(arr):
    """
    arr : numpy.array
        arr is (population, queen_size) shape
    """
    penalty = np.array([]).astype(int)
    for _arr in arr:
        penalty = np.append(arr=penalty,
                            values=get_penalty(_arr),
                            axis=None
                            )
    return penalty

In [7]:
penalty = evaluate_queen(arr=arr)
penalty

array([6, 4, 2, 0, 2, 4])

In [8]:
arr_computed_f = np.concatenate((arr,
                                 evaluate_queen(arr=arr).reshape(-1,1)),
                                axis=1)
arr_computed_f

array([[3, 4, 5, 2, 1, 6],
       [2, 5, 4, 3, 1, 4],
       [3, 5, 1, 4, 2, 2],
       [3, 5, 2, 4, 1, 0],
       [2, 4, 1, 5, 3, 2],
       [3, 5, 4, 1, 2, 4]])

## Tornament

In [9]:
def tournament(arr, times):
    output = np.empty(shape=(0, arr.shape[1])).astype(int)
    #print(output.shape)
    for i in range(times):
        idx = np.random.randint(arr.shape[0], size=2)
        #print(arr[idx,:].shape)
        output = np.append(arr=output,
                           values=arr[idx,:],
                           axis=0
                           )
    return output

In [10]:
# 2行ずつの束とみなす
arr_won_tournament = tournament(arr=arr_computed_f[:, :-1], times=3)
arr_won_tournament = np.concatenate((arr_won_tournament,
                                     evaluate_queen(arr=arr_won_tournament).reshape(-1,1)),
                                    axis=1)
arr_won_tournament

array([[3, 4, 5, 2, 1, 6],
       [3, 5, 1, 4, 2, 2],
       [2, 4, 1, 5, 3, 2],
       [3, 4, 5, 2, 1, 6],
       [3, 5, 2, 4, 1, 0],
       [2, 4, 1, 5, 3, 2]])

## Cross Over

In [11]:
np.random.randint(0, arr.shape[1] )

0

In [12]:
def single_point_crossover(arr):
    """
    arr : np.array
        array's row size should be even integer.(偶数)
    """
    times = int(arr.shape[0] / 2)
    idxes = []
    for i in range(times):
        idx = np.random.randint(0, arr.shape[1])
        idxes.append(idx)
        print("single point : ", idx)
        tmp = copy.deepcopy(arr[i*2, idx:])
        arr[i*2, idx:] = copy.deepcopy(arr[i*2+1, idx:])
        arr[i*2+1, idx:] = tmp
    return arr, idxes

In [13]:
arr_single_point_crossovered, idxes = single_point_crossover(arr_won_tournament[:, :-1])
arr_single_point_crossovered = np.concatenate((arr_single_point_crossovered,
                                               evaluate_queen(arr=arr_single_point_crossovered).reshape(-1,1)),
                                              axis=1)
arr_single_point_crossovered, idxes

single point :  0
single point :  2
single point :  0


(array([[3, 5, 1, 4, 2, 2],
        [3, 4, 5, 2, 1, 6],
        [2, 4, 5, 2, 1, 4],
        [3, 4, 1, 5, 3, 3],
        [2, 4, 1, 5, 3, 2],
        [3, 5, 2, 4, 1, 0]]), [0, 2, 0])

## OX crossover

In [14]:
def ox_crossover(arr, index):
    times = int(arr.shape[0] / 2)
    child_arr = np.empty(shape=(0, arr.shape[1])).astype(int)
    for i in range(times):
        _arr1 = np.empty(shape=(0,)).astype(int)
        _arr2 = np.append(arr=arr[i*2],
                          values=arr[i*2+1, index[i]:],
                          axis=None)
        for val in _arr2:
            if not val in _arr1:
                #_arr1 = np.insert(arr=_arr1, obj=-1, values=val)
                _arr1 = np.append(arr=_arr1,
                                  values=val,
                                  axis=None)
        child_arr = np.append(arr=child_arr,
                              values=_arr1[:arr.shape[1]].reshape(1,-1),
                              axis=0)
        _arr1 = np.empty(shape=(0,)).astype(int)
        _arr2 = np.append(arr=arr[i*2+1],
                          values=arr[i*2, index[i]:],
                          axis=None)
        for val in _arr2:
            if not val in _arr1:
                #_arr1 = np.insert(arr=_arr1, obj=-1, values=val)
                _arr1 = np.append(arr=_arr1,
                                  values=val,
                                  axis=None)
        child_arr = np.append(arr=child_arr,
                              values=_arr1[:arr.shape[1]].reshape(1,-1),
                              axis=0)
    return child_arr

In [15]:
arr_ox_crossovered = ox_crossover(arr=arr_single_point_crossovered[:, :-1], index=idxes)
arr_ox_crossovered = np.concatenate((arr_ox_crossovered,
                                     evaluate_queen(arr=arr_ox_crossovered).reshape(-1,1)),
                                    axis=1)
arr_ox_crossovered

array([[3, 5, 1, 4, 2, 2],
       [3, 4, 5, 2, 1, 6],
       [2, 4, 5, 1, 3, 2],
       [3, 4, 1, 5, 2, 2],
       [2, 4, 1, 5, 3, 2],
       [3, 5, 2, 4, 1, 0]])

## Mutations

In [16]:
def mutate(arr):
    for _arr in arr:
        idx = np.random.randint(low=0, high=arr.shape[1], size=2).astype(int)
        tmp = _arr[idx[0]]
        _arr[idx[0]] = _arr[idx[1]]
        _arr[idx[1]] = tmp
    return arr

In [17]:
arr_mutated = mutate(arr=arr_ox_crossovered[:, :-1])
arr_mutated = np.concatenate((arr_mutated,
                              evaluate_queen(arr=arr_mutated).reshape(-1,1)),
                             axis=1)
arr_mutated

array([[5, 3, 1, 4, 2, 0],
       [3, 2, 5, 4, 1, 4],
       [5, 4, 2, 1, 3, 2],
       [3, 4, 2, 5, 1, 2],
       [2, 4, 5, 1, 3, 2],
       [3, 2, 5, 4, 1, 4]])

# NextGeneration

In [18]:
class create_next_generation():
    def __init__(self):
        pass

    def create(self, arr2d):
        arr = copy.deepcopy(arr2d)
        self.arr_computed_f = np.concatenate((arr,
                                              self.evaluate_queen(arr2d=arr).reshape(-1,1)),
                                             axis=1)
        print("arr_computed_f\n", self.arr_computed_f)
        # 2行ずつの束とみなす
        self.arr_won_tournament = self.tournament(arr2d=self.arr_computed_f[:, :-1], times=3)
        self.arr_won_tournament = np.concatenate((self.arr_won_tournament,
                                                  self.evaluate_queen(arr2d=self.arr_won_tournament).reshape(-1,1)),
                                                 axis=1)
        print("won tornament\n", self.arr_won_tournament)
        self.arr_single_point_crossovered, idxes = self.single_point_crossover(self.arr_won_tournament[:, :-1])
        self.arr_single_point_crossovered = np.concatenate(
            (self.arr_single_point_crossovered,
             self.evaluate_queen(arr2d=self.arr_single_point_crossovered).reshape(-1,1)),
            axis=1)
        print("single point crossover\n", self.arr_single_point_crossovered)
        self.arr_ox_crossovered = self.ox_crossover(arr2d=self.arr_single_point_crossovered[:, :-1],
                                                    index=idxes)
        self.arr_ox_crossovered = np.concatenate((self.arr_ox_crossovered,
                                                  self.evaluate_queen(arr2d=self.arr_ox_crossovered).reshape(-1,1)),
                                                 axis=1)
        print("arr_ox_crossovered\n", self.arr_ox_crossovered)
        self.arr_mutated = self.mutate(arr2d=self.arr_ox_crossovered[:, :-1])
        self.arr_mutated = np.concatenate((self.arr_mutated,
                                           self.evaluate_queen(arr2d=self.arr_mutated).reshape(-1,1)),
                                          axis=1)
        print("arr_mutated\n", self.arr_mutated)
        return self.arr_mutated

    def evaluate_queen(self, arr2d):
        """
        arr : numpy.array
            arr is (population, queen_size) shape
        """
        arr = copy.deepcopy(arr2d)
        penalty = np.array([]).astype(int)
        for _arr in arr:
            penalty = np.append(arr=penalty,
                                values=self.get_penalty(_arr),
                                axis=None
                                )
        return penalty

    def get_penalty(self, arr1d):
        arr = copy.deepcopy(arr1d)
        penalty = 0
        for i in range(arr.shape[0]):
            for j in range(1, arr.shape[0]):
                if i+j < 5:  # 一番下を制限
                    if arr[i] - 1 + j < 5 and arr[i+j] == arr[i]+j:  # 右下
                        penalty += 1
                    if arr[i] - 1 - j >= 0 and arr[i+j] == arr[i]-j:  # 左下
                        penalty += 1
                if i-j >= 0:
                    if arr[i] - 1 + j < 5 and arr[i-j] == arr[i]+j:  # 右上
                        penalty += 1
                    if arr[i] - 1 - j >= 0 and arr[i-j] == arr[i]-j:  # 左上
                        penalty += 1
        penalty = int(penalty / 2)
        return penalty

    def tournament(self, arr2d, times):
        arr = copy.deepcopy(arr2d)
        output = np.empty(shape=(0, arr.shape[1])).astype(int)
        for i in range(times):
            idx = np.random.randint(arr.shape[0], size=2)
            output = np.append(arr=output,
                               values=arr[idx,:],
                               axis=0
                               )
        return output

    def single_point_crossover(self, arr2d):
        """
        arr : np.array
            array's row size should be even integer.(偶数)
        """
        arr = copy.deepcopy(arr2d)
        times = int(arr.shape[0] / 2)
        idxes = []
        for i in range(times):
            idx = np.random.randint(0, arr.shape[1])
            idxes.append(idx)
            print("single point : ", idx)
            tmp = copy.deepcopy(arr[i*2, idx:])
            arr[i*2, idx:] = copy.deepcopy(arr[i*2+1, idx:])
            arr[i*2+1, idx:] = tmp
        return arr, idxes

    def ox_crossover(self, arr2d, index):
        arr = copy.deepcopy(arr2d)
        times = int(arr.shape[0] / 2)
        child_arr = np.empty(shape=(0, arr.shape[1])).astype(int)
        for i in range(times):
            _arr1 = np.empty(shape=(0,)).astype(int)
            _arr2 = np.append(arr=arr[i*2],
                              values=arr[i*2+1, index[i]:],
                              axis=None)
            for val in _arr2:
                if not val in _arr1:
                    _arr1 = np.append(arr=_arr1,
                                      values=val,
                                      axis=None)
            child_arr = np.append(arr=child_arr,
                                  values=_arr1[:arr.shape[1]].reshape(1,-1),
                                  axis=0)
            _arr1 = np.empty(shape=(0,)).astype(int)
            _arr2 = np.append(arr=arr[i*2+1],
                              values=arr[i*2, index[i]:],
                              axis=None)
            for val in _arr2:
                if not val in _arr1:
                    _arr1 = np.append(arr=_arr1,
                                      values=val,
                                      axis=None)
            child_arr = np.append(arr=child_arr,
                                  values=_arr1[:arr.shape[1]].reshape(1,-1),
                                  axis=0)
        return child_arr

    def mutate(self, arr2d):
        arr = copy.deepcopy(arr2d)
        idxes = []
        for _arr in arr:
            idx = np.random.randint(low=0, high=arr.shape[1], size=2).astype(int)
            tmp = _arr[idx[0]]
            _arr[idx[0]] = _arr[idx[1]]
            _arr[idx[1]] = tmp
        return arr

# File Write

In [19]:
queen_size = 5
population = 6
arr = np.array([
    np.array([i for i in range(1,queen_size+1)])
    for j in range(population)
])
for i, _arr in enumerate(arr):
    np.random.shuffle(_arr)
    arr[i] = _arr
print(arr)

[[4 1 2 5 3]
 [4 2 1 3 5]
 [5 1 4 3 2]
 [3 2 5 4 1]
 [3 5 2 4 1]
 [3 2 4 1 5]]


In [20]:
fname = "queen.csv"
with open(fname, 'wt') as f:
    for i in range(2):
        instance = create_next_generation()
        arr = instance.create(arr)[:, :-1]
        f.write("Iteration : " + str(i+1) + "\n")
        f.write("Evaluate Function\n")
        f.write(",Queen Row\n")
        f.write("population,column1,column2,column3,column4,column5,EvaluateFunc\n")
        for idx, _arr in enumerate(instance.arr_computed_f):
            f.write("population"+str(idx+1)+"," + ','.join(["%.d" % val for val in _arr]) + "\n")
        f.write("Tournament\n")
        for idx, _arr in enumerate(instance.arr_won_tournament):
            f.write("population"+str(idx+1)+"," + ','.join(["%.d" % val for val in _arr]) + "\n")
        f.write("Single point crossover\n")
        for idx, _arr in enumerate(instance.arr_single_point_crossovered):
            f.write("population"+str(idx+1)+"," + ','.join(["%.d" % val for val in _arr]) + "\n")
        f.write("OX crossover\n")
        for idx, _arr in enumerate(instance.arr_ox_crossovered):
            f.write("population"+str(idx+1)+"," + ','.join(["%.d" % val for val in _arr]) + "\n")
        f.write("Mutations\n")
        for idx, _arr in enumerate(instance.arr_mutated):
            f.write("population"+str(idx+1)+"," + ','.join(["%.d" % val for val in _arr]) + "\n")
        f.write("\n")

arr_computed_f
 [[4 1 2 5 3 2]
 [4 2 1 3 5 2]
 [5 1 4 3 2 4]
 [3 2 5 4 1 4]
 [3 5 2 4 1 0]
 [3 2 4 1 5 2]]
won tornament
 [[3 2 5 4 1 4]
 [3 5 2 4 1 0]
 [3 5 2 4 1 0]
 [3 2 5 4 1 4]
 [5 1 4 3 2 4]
 [3 2 5 4 1 4]]
single point :  0
single point :  2
single point :  3
single point crossover
 [[3 5 2 4 1 0]
 [3 2 5 4 1 4]
 [3 5 5 4 1 2]
 [3 2 2 4 1 2]
 [5 1 4 4 1 1]
 [3 2 5 3 2 3]]
arr_ox_crossovered
 [[3 5 2 4 1 0]
 [3 2 5 4 1 4]
 [3 5 4 1 2 4]
 [3 2 4 1 5 2]
 [5 1 4 3 2 4]
 [3 2 5 4 1 4]]
arr_mutated
 [[3 5 1 4 2 2]
 [3 2 5 4 1 4]
 [3 5 4 1 2 4]
 [3 2 4 5 1 2]
 [1 5 4 3 2 6]
 [3 2 1 4 5 6]]
arr_computed_f
 [[3 5 1 4 2 2]
 [3 2 5 4 1 4]
 [3 5 4 1 2 4]
 [3 2 4 5 1 2]
 [1 5 4 3 2 6]
 [3 2 1 4 5 6]]
won tornament
 [[3 5 1 4 2 2]
 [3 5 1 4 2 2]
 [3 2 1 4 5 6]
 [3 2 5 4 1 4]
 [3 2 1 4 5 6]
 [3 5 4 1 2 4]]
single point :  0
single point :  2
single point :  0
single point crossover
 [[3 5 1 4 2 2]
 [3 5 1 4 2 2]
 [3 2 5 4 1 4]
 [3 2 1 4 5 6]
 [3 5 4 1 2 4]
 [3 2 1 4 5 6]]
arr_ox_crossovered
 [